In [1]:

%load_ext autoreload
%autoreload 2
from loaders.loaders import *
from dataset_dict import dataset_dict, output_path
from loaders.RecordDistributor import RecordDistributor


/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'1': 7826, '2': 6053, '3': 12775, '4': 5617, '0': 4325}


In [6]:
from datasets import load_dataset
ds= load_dataset("allenai/peS2o",streaming=True,split="train")
loader=iter(ds)

/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for allenai/peS2o contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/peS2o
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [23]:
next(loader)

{'added': '2018-04-03T04:06:09.115Z',
 'created': '1996-06-01T00:00:00.000Z',
 'id': '24962770',
 'source': 's2ag/train',
 'text': 'Two-phase helical CT for pancreatic tumors: pancreatic versus hepatic phase enhancement of tumor, pancreas, and vascular structures.\n\nPURPOSE\nTo quantitatively evaluate and validate a two-phase helical computed tomographic (CT) protocol for evaluation of pancreatic tumors.\n\n\nMATERIALS AND METHODS\nTwenty-seven patients with pathologically proved pancreatic adenocarcinomas prospectively underwent two-phase CT examination with helical acquisition during the pancreatic phase (40-70 seconds after infusion of intravenous contrast material at 3 mL/sec) and the hepatic phase (70-100 seconds after infusion). Mean CT attenuation values of tumor, bordering pancreas, and all major peripancreatic vessels were obtained for both time intervals.\n\n\nRESULTS\nMean tumor-pancreas contrast was significantly greater during the pancreatic phase (67 HU +/- 19) than the 